# Final Model

## Libraries

In [1]:
# main
import pandas as pd
import numpy as np
import time

# spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

## Setting everything for running the model
## Spotify API

In [2]:
import config

In [3]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))


## Scaler

In [4]:
spotify_playlist = pd.read_csv("../spotify_playlist.csv")
final_list = spotify_playlist.iloc[:, 2:]

In [5]:
# initilazing the scaler
scaler = StandardScaler()

# scaling the playlist
playlist_scaled = scaler.fit_transform(final_list)

## KMeans

In [6]:
# intilize the model
kmeans = KMeans(n_clusters=7, random_state=42)

# fit the model
kmeans.fit(playlist_scaled)

KMeans(n_clusters=7, random_state=42)

## Running the Final Model
### Loading the CSV Files

In [7]:
billboard = pd.read_csv("../top_hot_100.csv")
playlist = pd.read_csv("../playlist_with_clusters.csv")


## Testing the Model

In [8]:
#User inputs a song. If it's on the DF, it will return another one, if not it will search on Spotify.
def input_song(df):
    choice = input("Welcome to the IronCarla Song Recommender. Write down a song to get a recommendation: \n").lower()
    
    if choice in df.str.lower().values:
        return 1, choice
    else:
        return 0, choice


In [9]:
album, choice = input_song(billboard["album"])

if album == 1:
    suggestion = billboard.sample()
    suggestion_album = suggestion["album"].values.flatten()
    suggestion_artist = suggestion["artist(s)"].values.flatten()
    suggestion_album = str(suggestion_album).lstrip("[").rstrip("]")
    suggestion_artist = str(suggestion_artist).lstrip("[").rstrip("]")

    time.sleep(1.5)
    print("\nLoading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)
    print("We found something!\n")
    print(f"Here's our recommedation: {suggestion_album} from {suggestion_artist}. What a hit!")

elif album == 0:
    spoti_song = sp.search(q=choice)
        
    print(spoti_song["tracks"]["items"][0]["name"], " - ", spoti_song["tracks"]["items"][0]["artists"][0]["name"])
    song_features_spoti = pd.DataFrame(sp.audio_features(spoti_song["tracks"]["items"][0]["uri"]))
    spoti_features = song_features_spoti[["danceability", "energy", "mode", "acousticness", "instrumentalness", "liveness",
                          "loudness", "speechiness", "valence", "tempo"]]

    features_spoti_scaled = scaler.transform(spoti_features)
    spoti_cluster = kmeans.predict(features_spoti_scaled)
    match_song = playlist[playlist["cluster"] == int(spoti_cluster)]["song"]
    
    song_suggestion = match_song.sample()

    clean_song = list(song_suggestion)
    clean_song_2 = str(clean_song).lstrip("[").rstrip("]")    

    print("Wops, seems your song is not hot, but wait to get a recommendation based on the audio features of the song...\n")

    time.sleep(1.5)
    print("Loading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)

    print("Finally! Here's your perfect song:\n")
    print(f"{clean_song_2}\n")
    
        
    # Keep Working in that
    while True:
        redo = input(f"Do you want to predict another song? y/n: ")
        if redo == "y":
            print("Here's another recommendation:\n")
            spoti_suggestion = match_song.sample()
            clean_suggestion = list(spoti_suggestion)
            clean_suggestion_2 = str(clean_suggestion).lstrip("[").rstrip("]")
            print(f"{clean_suggestion_2}\n")
            
        elif redo == "n":
            print("\nThanks for using IronCarla Song Recommender")
            break
        else:
            print("Sorry, we couldn't detect your choice. Remember to write 'y' or 'n'")


Welcome to the IronCarla Song Recommender. Write down a song to get a recommendation: 
danza kuduro
Danza Kuduro  -  Don Omar
Wops, seems your song is not hot right now! Let me find you something else.

Loading..
Loading...

Finally! Here's your perfect song:

'Mona Lisa'

Do you want to predict another song? y/n: y
Here's another recommendation:

'Sunday Candy'

Do you want to predict another song? y/n: n

Thanks for using IronCarla Song Recommender
